In [ ]:
import os
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import defaultdict
nltk.download('punkt')
!pip install ipywidgets
from ipywidgets import interact
from IPython.display import display

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/InfoRetrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/InfoRetrive


In [ ]:
#to read stop words
def read_stop_words(stop_words_file):
    with open(stop_words_file, 'r') as f:
        stop_words = set(f.read().splitlines())
    return stop_words

stop_words_file = "/content/drive/MyDrive/InfoRetrive/Stopword-List.txt"

stop_words = read_stop_words(stop_words_file)
print(stop_words)






{'', 'in', 'and ', 'can', 'is ', 'all', 'at', 'had', 'to', 'on', 'up', 'once ', 'his', 'for', 'have', 'we', 'no', 'her', 'has', 'of ', 'do', 'are', 'be', 'a', 'am', 'the', 'as'}


In [ ]:
ps = PorterStemmer()

# extract tokens from multiple documents in a folder
def create_positional_index_from_folder(folder_path):
    positional_index = defaultdict(set) #using set instead of list to make sure no dublicates

    for filename in os.listdir(folder_path):  # Iterate over each document
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='cp1252') as file:  # cp1252 is windows text file encoding
          text = file.read()
          tokens =  word_tokenize(text.lower())  #from nlkt module

          for position, token in enumerate(tokens): #if tokens not in stop_words:
            if token not in stop_words:  # Check if token is not a stop word
                positional_index[token].add((filename, position))  #(27527 tokens with no stemming)
                #stemmed_token = ps.stem(token)  # Stem the token  (23778 toekns)


    return positional_index

In [ ]:

#main
folder_path = "/content/drive/MyDrive/InfoRetrive/ResearchPapers"
positional_index = create_positional_index_from_folder(folder_path)
# for token, positions in positional_index.items():
#     print(f"{token}: {positions}")

print("Length of the inverted index:", len(positional_index))



Length of the inverted index: 27527


-----------------------------------------------

In [ ]:
def proximity_query(positional_index, term1, term2, max_distance):
    if term1 not in positional_index or term2 not in positional_index: #if term dosent exist
        return []

    term1_positions = positional_index[term1]
    term2_positions = positional_index[term2]

    results = []

    for doc1, pos1 in term1_positions:
        for doc2, pos2 in term2_positions:
            if doc1 == doc2: #if both terms in same documnt
                if abs(pos1 - pos2) <= max_distance: #calculate and check difference
                    results.append(doc1) #if within the max distance
                    break  # Break as soon as one valid occurrence is found in the document

    return results


# results = proximity_query(positional_index, "past", "research", 4)
# print("Result", results)


Result ['12.txt']


In [ ]:
def search(query):
    try:

        query_parts = query.split('/')
        terms_distance = query_parts[0].split() #splits into 2 words by white space
        term1 = terms_distance[0] #term1
        term2 = terms_distance[1] #term2
        max_distance = int(query_parts[1].strip()) #max distance and remove trailing white space

        results = proximity_query(positional_index, term1, term2, max_distance)
        print("Results:", results)
    except Exception as e:
        print("Error:", e)

interact(search, query=" ");

interactive(children=(Text(value=' ', description='query'), Output()), _dom_classes=('widget-interact',))